# Import file


In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import glob
import cv2
import os
from natsort import natsorted
from sklearn.preprocessing import MinMaxScaler
from scipy.signal import convolve2d
from skimage.transform import resize
from skimage.io import imread
import openpyxl
from tqdm import trange

np.set_printoptions(suppress=True)

path = '/Users/tae/Desktop/Project/image/*'

minmax= MinMaxScaler()


size_columns = 100

In [3]:

def readfile():
    folder_array =  [natsorted(glob.glob(i+'/*')) for i in glob.glob(path)]
    return  folder_array

def rgb2gray(folders):
    return [[cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2GRAY)/255 for path in folder] for folder in folders]
    


def convolution(images):
    images = np.array(images)
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])

    image = images
    while True:
        if image.shape[0] < 3 or image.shape[1] < 3:
            break
            
        new_image = convolve2d(image, kernel, mode='valid')
        
        if new_image.shape[0] <= 1 or new_image.shape[1] <= 1 or new_image.size <= 100:
            break
        image = new_image

    final_image = image

    return final_image.flatten()[:100]



def exportExcel(dataset):
    with pd.ExcelWriter('dataset100_unnormalize.xlsx') as writer:
        for index, data in enumerate(dataset):
            
            df=pd.DataFrame(data)
            sheet_name = f'Sheet{index+1}'
            df.to_excel(writer,sheet_name=sheet_name , header=None , index=None)
            
    return 'Export Success'



In [5]:
# Main code default

folders = readfile()
folder_gray = rgb2gray(folders)

result = []

for folder in folder_gray:
    convolved_images = []
    for idx_img in trange(len(folder), desc="Convolving Images"):
        convolved_image = convolution(folder[idx_img])
        convolved_images.append(convolved_image)
    result.append(convolved_images)

# result = [scaler.fit_transform(nml) for nml in result]

exportExcel(result)

Convolving Images: 100%|██████████████████████| 423/423 [00:23<00:00, 18.04it/s]


'Export Success'

In [ ]:
# Convolution from torch 512

from tqdm import trange
from img2vec_pytorch import Img2Vec
from PIL import Image

def convolution_from_torch(path):
    img = Image.open(path)
    image = Img2Vec().get_vec(img)
    return image

all_file = readfile()
dataset = []
for i in range(4):
    data = []
    for path in trange(len(all_file[i])):
        data.append(convolution_from_torch(all_file[i][path]))
    dataset.append(data)
exportExcel(dataset)